##Tutoriel: Les outils de base de Private Deep Learning

Bienvenue dans le didacticiel d'introduction de PySyft pour un apprentissage approfondi décentralisé préservant la confidentialité. Cette série de cahiers est un guide étape par étape pour vous familiariser avec les nouveaux outils et techniques nécessaires pour effectuer un apprentissage approfondi sur des données / modèles secrets / privés sans les centraliser sous une seule autorité.

**Portée**: notez que nous ne parlerons pas seulement de la manière de décentraliser / crypter les données, mais de la façon dont PySyft peut être utilisé pour aider à décentraliser l'ensemble de l'écosystème autour des données, y compris les bases de données où les données sont stockées et interrogées, et les modèles neuronaux qui sont utilisés pour extraire des informations des données. À mesure que de nouvelles extensions de PySyft sont créées, ces blocs-notes seront étendus avec de nouveaux didacticiels pour expliquer la nouvelle fonctionnalité.

Auteurs:

* Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)

##Contour:

* Partie 1: Les outils de base de Private Deep Learning

##Pourquoi suivre ce didacticiel?

1) Un avantage de carrière compétitif - Au cours des 20 dernières années, la révolution numérique a rendu les données de plus en plus accessibles en quantités toujours plus importantes à mesure que les processus analogiques se sont numérisés. Cependant, avec la nouvelle réglementation telle que le GDPR, les entreprises sont sous pression pour avoir moins de liberté dans la façon dont elles utilisent - et plus important encore dans la façon dont elles analysent - les informations personnelles. Bottom Line: Les Data Scientists n'auront pas accès à autant de données avec des outils "old school", mais en apprenant les outils du Private Deep Learning, VOUS pouvez être en avance sur cette courbe et avoir un avantage compétitif dans votre carrière.

2) Opportunités entrepreneuriales - Il existe toute une série de problèmes dans la société que le Deep Learning peut résoudre, mais bon nombre des plus importants n'ont pas été explorés car cela nécessiterait l'accès à des informations incroyablement sensibles sur les personnes (pensez à utiliser le Deep Learning pour aider les gens avec des problèmes mentaux ou relationnels!). Ainsi, l'apprentissage Private Deep Learning ouvre pour vous une multitude de nouvelles opportunités de démarrage qui n'étaient pas disponibles auparavant pour d'autres sans ces outils.

3) Bien social - Le Deep Learning peut être utilisé pour résoudre une grande variété de problèmes dans le monde réel, mais le Deep Learning sur les informations personnelles est le Deep Learning sur les gens, pour les gens. Apprendre à faire du Deep Learning sur des données que vous ne possédez pas représente plus qu'une carrière ou une opportunité entrepreneuriale, c'est l'occasion d'aider à résoudre certains des problèmes les plus personnels et les plus importants de la vie des gens - et de le faire à grande échelle.


##Comment obtenir un crédit supplémentaire?

* Star PySyft sur GitHub! - https://github.com/OpenMined/PySyft
* Faites une vidéo Youtube enseignant ce cahier!

##Partie -1: Conditions préalables

* Connaissez PyTorch - sinon, suivez le cours http://fast.ai et revenez
* Lisez le document cadre PySyft https://arxiv.org/pdf/1811.04017.pdf! Cela vous donnera un aperçu complet de la façon dont PySyft est construit, ce qui aidera les choses à avoir plus de sens.

##Partie 0: 
Pour commencer, vous devrez vous assurer que les bonnes choses sont installées. Pour ce faire, rendez-vous sur le fichier Readme de PySyft et suivez les instructions de configuration. TLDR pour la plupart des gens est.

* Installez Python 3.6 ou supérieur
* Installez PyTorch 1.3
* Clone PySyft (git clone https://github.com/OpenMined/PySyft.git)
* cd PySyft
* pip install -r pip-dep/requirements.txt
* pip install -r pip-dep/requirements_udacity.txt
* python setup.py install udacity
* python setup.py test




Si une partie de cela ne fonctionne pas pour vous (ou l'un des tests échoue) - consultez d'abord le fichier [README](https://github.com/OpenMined/PySyft) pour l'aide à l'installation, puis ouvrez un problème GitHub ou envoyez une requête ping au canal #beginner dans notre mou! [slack.openmined.org](slack.openmined.org)




In [0]:
# Exécutez cette cellule pour voir si les choses fonctionnent
import sys

import torch
from torch.nn import Parameter
import torch.nn as nn
import torch.nn.functional as F

import syft as sy
hook = sy.TorchHook(torch)

torch.tensor([1,2,3,4,5])

Si cette cellule est exécutée, alors c'est parti pour les courses! Faisons cela!

##Partie 1 - Les outils de base de la science des données privée et décentralisée

Donc - la première question que vous vous posez peut-être est la suivante: comment diable formons-nous un modèle sur des données auxquelles nous n'avons pas accès?

Eh bien, la réponse est étonnamment simple. Si vous avez l'habitude de travailler dans PyTorch, alors vous avez l'habitude de travailler avec des objets torche.Tensor comme ceux-ci!

In [0]:
x = torch.tensor([1,2,3,4,5])
y = x + x
print(y)


Évidemment, l'utilisation de ces Tensors super sophistiqués (et puissants!) Est importante, mais vous oblige également à avoir les données sur votre machine locale. C'est là que commence notre voyage.

##Section 1.1 - Envoi de tenseurs à la machine de Bob

Alors que normalement nous effectuerions la science des données / l'apprentissage en profondeur sur la machine qui contient les données, nous voulons maintenant effectuer ce type de calcul sur une autre machine. Plus précisément, nous ne pouvons plus simplement supposer que les données se trouvent sur notre machine locale.

Ainsi, au lieu d'utiliser des tenseurs Torch, nous allons maintenant travailler avec des pointeurs vers des tenseurs. Permettez-moi de vous montrer ce que je veux dire. Créons d'abord une machine à "faire semblant" appartenant à une personne "à faire semblant" - nous l'appellerons Bob.


In [0]:
bob = sy.VirtualWorker(hook, id="bob")

Disons que la machine de Bob est sur une autre planète - peut-être sur Mars! Mais, pour le moment, la machine est vide. Créons des données pour que nous puissions les envoyer à Bob et en apprendre davantage sur les pointeurs!

In [0]:
x = torch.tensor([1,2,3,4,5])
y = torch.tensor([1,1,1,1,1])

Et maintenant - envoyons nos tensors à Bob !!


In [0]:
x_ptr = x.send(bob)
y_ptr = y.send(bob)

In [0]:
x_ptr

BOOM! Maintenant, Bob a deux tensors! Vous ne me croyez pas? Jetez un œil par vous-même!

In [0]:
bob._objects

In [0]:
z = x_ptr + x_ptr

In [0]:
z

In [0]:
bob._objects

Maintenant remarquez quelque chose. Lorsque nous avons appelé `x.send (bob)`, il a renvoyé un nouvel objet que nous avons appelé `x_ptr`. Ceci est notre premier pointeur sur un tensor. Les pointeurs vers les tenseurs ne contiennent PAS réellement les données eux-mêmes. Au lieu de cela, ils contiennent simplement des métadonnées sur un tensor (avec des données) stockées sur une autre machine. Le but de ces tenseurs est de nous donner une API intuitive pour dire à l'autre machine de calculer les fonctions utilisant ce tensor. Jetons un coup d'œil aux métadonnées que contiennent les pointeurs.

In [0]:
x_ptr

Découvrez ces métadonnées!

Il existe deux principaux attributs spécifiques aux pointeurs:

* x_ptr.location: bob, l'emplacement, une référence à l'emplacement vers lequel pointe le pointeur
* x_ptr.id_at_location: <entier aléatoire>, l'id où le tensor est stocké à l'emplacement

Ils sont imprimés au format `<id_at_location> @ <location>`

Il existe également d'autres attributs plus génériques:

* x_ptr.id: <entier aléatoire>, l'id de notre tensor de pointeur, il a été alloué de façon aléatoire
* x_ptr.owner: "moi", l'ouvrier qui possède le tensor de pointeur, ici c'est l'ouvrier local, nommé "moi"

In [0]:
x_ptr.location

In [0]:
bob

In [0]:
bob == x_ptr.location

In [0]:
x_ptr.id_at_location

In [0]:
x_ptr.owner

Vous vous demandez peut-être pourquoi le travailleur local qui possède le pointeur est également un VirtualWorker, même si nous ne l'avons pas créé. Fait amusant, tout comme nous avions un objet VirtualWorker pour Bob, nous (par défaut) en avons toujours un pour nous également. Ce travailleur est automatiquement créé lorsque nous avons appelé `hook = sy.TorchHook()` et vous n'avez donc généralement pas à le créer vous-même.


In [0]:
me = sy.local_worker
me

In [0]:
me == x_ptr.owner

Et enfin, tout comme nous pouvons appeler `.send()` sur un tenseur, nous pouvons appeler `.get()` sur un pointeur vers un tenseur pour le récupérer !!!

In [0]:
x_ptr

In [0]:
x_ptr.get()

In [0]:
y_ptr

In [0]:
y_ptr.get()

In [0]:
z.get()

In [0]:
bob._objects

Et comme vous pouvez le voir ... Bob n'a plus les tensors !!! Ils sont revenus sur notre machine!

##Section 1.2 - Utilisation des pointeurs tensors

Donc, envoyer et recevoir des tensors de Bob est génial, mais ce n'est pas du Deep Learning! Nous voulons pouvoir effectuer des opérations de tensors sur des tensors distants. Heureusement, les pointeurs tensors rendent cela assez facile! Vous pouvez simplement utiliser des pointeurs comme vous le feriez avec des tensors normaux!

In [0]:
x = torch.tensor([1,2,3,4,5]).send(bob)
y = torch.tensor([1,1,1,1,1]).send(bob)

In [0]:
z = x + y

In [0]:
z

Et voilà!

Dans les coulisses, quelque chose de très puissant s'est produit. Au lieu que x et y calculent un ajout localement, une commande a été sérialisée et envoyée à Bob, qui a effectué le calcul, créé un tensor z, puis nous a renvoyé le pointeur z!

Si nous appelons `.get()` sur le pointeur, nous recevrons alors le résultat sur notre machine!

In [0]:
z.get()

##Fonctions de Torch 

Cette API a été étendue à toutes les opérations de Torch !!!

In [0]:
x

In [0]:
y

In [0]:
z = torch.add(x,y)
z

In [0]:
z.get()

##Variables(comprenant backpropagation!)

In [0]:
x = torch.tensor([1,2,3,4,5.], requires_grad=True).send(bob)
y = torch.tensor([1,1,1,1,1.], requires_grad=True).send(bob)

In [0]:
z = (x + y).sum()

In [0]:
z.backward()

In [0]:
x = x.get()

In [0]:
x

In [0]:
x.grad

Donc, comme vous pouvez le voir, l'API est vraiment assez flexible et capable d'effectuer presque toutes les opérations que vous effectuez normalement dans Torch sur des données distantes. Cela jette les bases de nos protocoles plus avancés de préservation de la confidentialité tels que Le Federated Learning, Secure Multi-Party Computation, and Differential Privacy !

##Toutes nos félicitations!!! - Il est temps de rejoindre la communauté!
Félicitations pour avoir terminé ce didacticiel pour ordinateur portable! Si cela vous a plu et que vous souhaitez rejoindre le mouvement vers la préservation de la vie privée, la propriété décentralisée de l'AI et la chaîne d'approvisionnement de l'AI (données), vous pouvez le faire de la manière suivante!

##Star PySyft sur GitHub
Le moyen le plus simple d'aider notre communauté est simplement de mettre en vedette les dépôts GitHub! Cela permet de faire connaître les outils sympas que nous construisons.

* [Star PySyft](https://github.com/OpenMined/PySyft)

##Joindrez notre Slack!
La meilleure façon de vous tenir au courant des dernières avancées est de rejoindre notre communauté! Vous pouvez le faire en remplissant le formulaire à http://slack.openmined.org

##Joindrez un projet de code!
La meilleure façon de contribuer à notre communauté est de devenir un contributeur de code! À tout moment, vous pouvez accéder à la page Problèmes de PySyft GitHub et filtrer pour "Projects". Cela vous montrera tous les billets de haut niveau donnant un aperçu des projets que vous pouvez rejoindre! Si vous ne souhaitez pas rejoindre un projet, mais que vous souhaitez faire un peu de codage, vous pouvez également rechercher d'autres mini-projets "uniques" en recherchant les problèmes GitHub marqués "good first issue".
* [PySyft Project](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
* [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

##Faire un don
Si vous n'avez pas le temps de contribuer à notre base de code, mais souhaitez tout de même apporter votre soutien, vous pouvez également devenir Backer sur notre Open Collective. Tous les dons vont à notre hébergement Web et à d'autres dépenses de la communauté telles que les hackathons et les rencontres!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)